<a href="https://colab.research.google.com/github/Adebesindedayo/Women-s-Mentorship-1-Hack-for-Safety-by-Agence-Fran-aise-de-D-veloppement/blob/main/Women's_Mentorship_1_Hack_for_Safety_by_Agence_Fran%C3%A7aise_de_D%C3%A9veloppement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

ModuleNotFoundError: No module named 'google'

In [ ]:
pip install catboost

In [ ]:
!pip install rgf_python


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as sc
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)
from lightgbm import LGBMClassifier
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from rgf.sklearn import RGFClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier,GradientBoostingClassifier
from sklearn.model_selection import KFold,train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, plot_confusion_matrix, plot_roc_curve
from sklearn.preprocessing import LabelEncoder
sns.set()
pd.set_option('display.max_columns', None)


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

# GETTING ALL THE DATA SET

In [ ]:
train= pd.read_csv("/content/gdrive/My Drive/FIRST_HACKATON/Train.csv")
train.head()


In [ ]:
test= pd.read_csv("/content/gdrive/My Drive/FIRST_HACKATON/Test.csv")
test.head()


 ## BASIC EXPLORATORY DATA ANALYSIS


In [ ]:
train.describe(include='all')

In [ ]:
train.info()
#all except the target variable is a categorical data

In [ ]:
train.columns

In [ ]:
train.shape

**## HANDLING OF CATEGORICAL VARIABLE OF THE TRAIN AND TEST DATA SET USING LABEL ENCODING**

In [ ]:
le = LabelEncoder()

In [3]:
train.gender = le.fit_transform(train.gender).astype(np.uint8)
test.gender = le.fit_transform(test.gender).astype(np.uint8)

train.race = le.fit_transform(train.race).astype(np.uint8)
test.race = le.fit_transform(test.race).astype(np.uint8)

train.dwelling = le.fit_transform(train.dwelling).astype(np.uint8)
test.dwelling = le.fit_transform(test.dwelling).astype(np.uint8)

train.dwelling_type = le.fit_transform(train.dwelling_type).astype(np.uint8)
test.dwelling_type = le.fit_transform(test.dwelling_type).astype(np.uint8)

train.province_code = le.fit_transform(train.province_code).astype(np.uint8)
test.province_code = le.fit_transform(test.province_code).astype(np.uint8)

train.metro_code = le.fit_transform(train.metro_code).astype(np.uint8)
test.metro_code = le.fit_transform(test.metro_code).astype(np.uint8)

train.nationality= le.fit_transform(train.nationality	).astype(np.uint8)
test.nationality = le.fit_transform(test.nationality	).astype(np.uint8)

train.RTH= le.fit_transform(train.RTH).astype(np.uint8)
test.RTH= le.fit_transform(test.RTH).astype(np.uint8)

train.marital_st= le.fit_transform(train.marital_st).astype(np.uint8)
test.marital_st= le.fit_transform(test.marital_st).astype(np.uint8)

train.Lang_inside	= le.fit_transform(train.Lang_inside	).astype(np.uint8)
test.Lang_inside= le.fit_transform(test.Lang_inside	).astype(np.uint8)

train.Lang_outside= le.fit_transform(train.Lang_outside	).astype(np.uint8)
test.Lang_outside	= le.fit_transform(test.Lang_outside).astype(np.uint8)

train.Education= le.fit_transform(train.Education).astype(np.uint8)
test.Education= le.fit_transform(test.Education).astype(np.uint8)

train.nature_of_work= le.fit_transform(train.nature_of_work	).astype(np.uint8)
test.nature_of_work= le.fit_transform(test.nature_of_work	).astype(np.uint8)

train.lw_business= le.fit_transform(train.lw_business).astype(np.uint8)
test.lw_business= le.fit_transform(test.lw_business	).astype(np.uint8)

train.lw_work= le.fit_transform(train.lw_work).astype(np.uint8)
test.lw_work= le.fit_transform(test.lw_work).astype(np.uint8)

train.help_on_household	= le.fit_transform(train.help_on_household).astype(np.uint8)
test.help_on_household= le.fit_transform(test.help_on_household).astype(np.uint8)

train.job_or_business= le.fit_transform(train.job_or_business).astype(np.uint8)
test.job_or_business= le.fit_transform(test.job_or_business).astype(np.uint8)

NameError: name 'le' is not defined

**##  SETTING THE FEATURES AND THE LABEL AS X AND Y RESPECTIVELY**

In [ ]:
X = train.drop(['ID', 'target'], axis =1)
y = train.target

**DATA VISUALIZATION OF THE TARGET VARIABLE**

In [ ]:
sns.countplot(train["target"])

**THERE IS CLASS IMBALANCE WITH THE TARGET VARIABLE**

In [ ]:
X[y == 1]

**## DEALING WITH CLASS IMBALANCE USING RESAMPLING**

In [ ]:
print('Number of class 1 samples before:',X[y == 1].shape[0])

In [ ]:
print('Number of class 0 samples before:',X[y == 0].shape[0])

In [ ]:
X_upsampled, y_upsampled = resample(X[y == 1],y[y == 1],replace=True,n_samples=X[y == 0].shape[0],random_state=123)
print('Number of class 1 samples after:',X_upsampled.shape[0])

In [ ]:
X_bal = np.vstack((X[y == 0], X_upsampled))
y_bal = np.hstack((y[y == 0], y_upsampled))
y_pred = np.zeros(y_bal.shape[0])
np.mean(y_pred == y_bal) * 100

In [ ]:
# for seed value 45

preds, scores = [], []
 
seed = 64
 
Vc = VotingClassifier(estimators=[
('cat',  CatBoostClassifier(verbose=False, thread_count=-1, random_state=seed)), 
('rgf', RGFClassifier()), ("rgf1", RGFClassifier(algorithm="RGF_Sib")), ('cat1', CatBoostClassifier(bagging_temperature = 0.8609895558337871, border_count=116, depth=5, iterations=270, l2_leaf_reg=3, 
                         learning_rate=0.07085617391173239, random_strength=0.08820672130534879, random_state=seed, scale_pos_weight=0.390675124341283, verbose=False, thread_count=-1)),
                           
('gr' ,GradientBoostingClassifier()),
('rg',RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=seed)), ('rg1', RandomForestClassifier(n_estimators=700, n_jobs=-1, random_state=seed))], 
voting="soft", weights=[3, 1.5,1.5,1,1,1,1])
 
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed) 

X = train.drop(['ID', 'target'], axis =1)
y = train.target
test_x=test.drop('ID', axis =1)
 
i = 5
for train_index, test_index in skf.split(X, y):
  print("Number of splits left {0} to train.".format(i))
  x_train, x_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
  pipe = Pipeline([("SC", MinMaxScaler()), ("clf", Vc)])
  pipe.fit(x_train, y_train)
  score = roc_auc_score(y_test, pipe.predict_proba(x_test)[:, 1])
  scores.append(score) 
  pred = pipe.predict_proba(test_x)[:, 1]
  preds.append(pred)
  print(score)
  i -= 1
 
np.mean(scores)

In [ ]:
ss = pd.read_csv("/content/gdrive/My Drive/FIRST_HACKATON/SampleSubmission.csv")
p = np.mean(preds, axis=0)
ss["target"] = p
ss.to_csv("zindi13.csv", index=False)